In [2]:
import pandas as pd
import re
import numpy as np
import nltk
import string
import fasttext
import contractions
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
plt.xticks(rotation=70)
pd.options.mode.chained_assignment = None
pd.set_option('display.max_colwidth', 100)
%matplotlib inline

In [3]:
data = pd.read_csv('F:/Catatan Materi/Semester 7/TA 1/Data_set/Training/train.csv')

In [4]:
data[:10]

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Luc...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart",Daniel J. Flynn,Ever get the feeling your life circles the roundabout rather than heads in a straight line towar...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, 2016 \nThe tension between intelligence analysts a...",1
3,3,15 Civilians Killed In Single US Airstrike Have Been Identified,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstrike Have Been Identified The rate at which civilia...,1
4,4,Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery,Howard Portnoy,Print \nAn Iranian woman has been sentenced to six years in prison after Iran’s Revolutionary Gu...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He Bombed North Korea over Lack of Trans Bathrooms (...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voice of Reason. [In this week’s exclusive clip for B...",0
6,6,"Life: Life Of Luxury: Elton John’s 6 Favorite Shark Pictures To Stare At During Long, Transconti...",NaN,Ever wonder how Britain’s most iconic pop pianist gets through a long flight? Here are the six p...,1
7,7,Benoît Hamon Wins French Socialist Party’s Presidential Nomination - The New York Times,Alissa J. Rubin,"PARIS — France chose an idealistic, traditional candidate in Sunday’s primary to represent ...",0
8,8,Excerpts From a Draft Script for Donald Trump’s Q&ampA With a Black Church’s Pastor - The New Yo...,NaN,Donald J. Trump is scheduled to make a highly anticipated visit to an church in Detroit on Sat...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Courtesy of Trump Associates - The New York Times",Megan Twohey and Scott Shane,"A week before Michael T. Flynn resigned as national security adviser, a sealed proposal was to...",0


In [5]:
data.shape

(20800, 5)

In [6]:
#mengambil atribut pada kolom 2-4 (1:4) (menggunakan rentang/interval)
#att_train = data_train.iloc[:, 1:4]
#mengambil atribut "title" dan "text" (menggunakan nama atribut)
data2 =data.loc[:, ['id','title', 'label']]

In [7]:
data2[:10]

,id,title,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart",0
2,2,Why the Truth Might Get You Fired,1
3,3,15 Civilians Killed In Single US Airstrike Have Been Identified,1
4,4,Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery,1
5,5,Jackie Mason: Hollywood Would Love Trump if He Bombed North Korea over Lack of Trans Bathrooms (...,0
6,6,"Life: Life Of Luxury: Elton John’s 6 Favorite Shark Pictures To Stare At During Long, Transconti...",1
7,7,Benoît Hamon Wins French Socialist Party’s Presidential Nomination - The New York Times,0
8,8,Excerpts From a Draft Script for Donald Trump’s Q&ampA With a Black Church’s Pastor - The New Yo...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Courtesy of Trump Associates - The New York Times",0


In [8]:
#mengecek nilai null pada kolom id dan title dan label
for col in data2.columns:
    print(col, data2[col].isnull().sum())

id 0
title 558
label 0


In [9]:
data3 = data2.dropna()

In [10]:
#mengecek nilai null pada kolom id dan title dan label
for col in data3.columns:
    print(col, data3[col].isnull().sum())

id 0
title 0
label 0


In [11]:
data3.shape

(20242, 3)

In [12]:
#menghapus angka
data3['title'] = data3['title'].str.replace('\d+', '')

In [13]:
data3[:10]

,id,title,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart",0
2,2,Why the Truth Might Get You Fired,1
3,3,Civilians Killed In Single US Airstrike Have Been Identified,1
4,4,Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery,1
5,5,Jackie Mason: Hollywood Would Love Trump if He Bombed North Korea over Lack of Trans Bathrooms (...,0
6,6,"Life: Life Of Luxury: Elton John’s Favorite Shark Pictures To Stare At During Long, Transcontin...",1
7,7,Benoît Hamon Wins French Socialist Party’s Presidential Nomination - The New York Times,0
8,8,Excerpts From a Draft Script for Donald Trump’s Q&ampA With a Black Church’s Pastor - The New Yo...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Courtesy of Trump Associates - The New York Times",0


In [14]:
#melakukan contraction (mengembalikan singkatan ke bentuk awalnya) contoh: you're => you are
data3['no_contract'] = data3['title'].apply(lambda x: 
                                                    [contractions.fix(word) for word in x.split()])

In [15]:
data3[:10]

,id,title,label,no_contract
0,0,House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It,1,"[House, Dem, Aide:, We, did not, Even, See, Comey’s, Letter, Until, Jason, Chaffetz, Tweeted, It]"
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart",0,"[FLYNN:, Hillary, Clinton,, Big, Woman, on, Campus, -, Breitbart]"
2,2,Why the Truth Might Get You Fired,1,"[Why, the, Truth, Might, Get, You, Fired]"
3,3,Civilians Killed In Single US Airstrike Have Been Identified,1,"[Civilians, Killed, In, Single, US, Airstrike, Have, Been, Identified]"
4,4,Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery,1,"[Iranian, woman, jailed, for, fictional, unpublished, story, about, woman, stoned, to, death, fo..."
5,5,Jackie Mason: Hollywood Would Love Trump if He Bombed North Korea over Lack of Trans Bathrooms (...,0,"[Jackie, Mason:, Hollywood, Would, Love, Trump, if, He, Bombed, North, Korea, over, Lack, of, Tr..."
6,6,"Life: Life Of Luxury: Elton John’s Favorite Shark Pictures To Stare At During Long, Transcontin...",1,"[Life:, Life, Of, Luxury:, Elton, John’s, Favorite, Shark, Pictures, To, Stare, At, During, Long..."
7,7,Benoît Hamon Wins French Socialist Party’s Presidential Nomination - The New York Times,0,"[Benoît, Hamon, Wins, French, Socialist, Party’s, Presidential, Nomination, -, The, New, York, T..."
8,8,Excerpts From a Draft Script for Donald Trump’s Q&ampA With a Black Church’s Pastor - The New Yo...,0,"[Excerpts, From, a, Draft, Script, for, Donald, Trump’s, Q&ampA, With, a, Black, Church’s, Pasto..."
9,9,"A Back-Channel Plan for Ukraine and Russia, Courtesy of Trump Associates - The New York Times",0,"[A, Back-Channel, Plan, for, Ukraine, and, Russia,, Courtesy, of, Trump, Associates, -, The, New..."


In [16]:
#mengembalikan kembali teks yang dispit dan ditampung oleh attribut title_str
data3['title_str'] = [' '.join(map(str, l)) for l in data3['no_contract']]
data3.head()

,id,title,label,no_contract,title_str
0,0,House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It,1,"[House, Dem, Aide:, We, did not, Even, See, Comey’s, Letter, Until, Jason, Chaffetz, Tweeted, It]",House Dem Aide: We did not Even See Comey’s Letter Until Jason Chaffetz Tweeted It
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart",0,"[FLYNN:, Hillary, Clinton,, Big, Woman, on, Campus, -, Breitbart]","FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart"
2,2,Why the Truth Might Get You Fired,1,"[Why, the, Truth, Might, Get, You, Fired]",Why the Truth Might Get You Fired
3,3,Civilians Killed In Single US Airstrike Have Been Identified,1,"[Civilians, Killed, In, Single, US, Airstrike, Have, Been, Identified]",Civilians Killed In Single US Airstrike Have Been Identified
4,4,Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery,1,"[Iranian, woman, jailed, for, fictional, unpublished, story, about, woman, stoned, to, death, fo...",Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery


# Tokenization

In [17]:
#melakukan tokenisasi pada atribut "title_str" dan ditampung atribut baru "tokenized"
data3['tokenized'] = data3['title_str'].apply(word_tokenize)
data3.head()

,id,title,label,no_contract,title_str,tokenized
0,0,House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It,1,"[House, Dem, Aide:, We, did not, Even, See, Comey’s, Letter, Until, Jason, Chaffetz, Tweeted, It]",House Dem Aide: We did not Even See Comey’s Letter Until Jason Chaffetz Tweeted It,"[House, Dem, Aide, :, We, did, not, Even, See, Comey, ’, s, Letter, Until, Jason, Chaffetz, Twee..."
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart",0,"[FLYNN:, Hillary, Clinton,, Big, Woman, on, Campus, -, Breitbart]","FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart","[FLYNN, :, Hillary, Clinton, ,, Big, Woman, on, Campus, -, Breitbart]"
2,2,Why the Truth Might Get You Fired,1,"[Why, the, Truth, Might, Get, You, Fired]",Why the Truth Might Get You Fired,"[Why, the, Truth, Might, Get, You, Fired]"
3,3,Civilians Killed In Single US Airstrike Have Been Identified,1,"[Civilians, Killed, In, Single, US, Airstrike, Have, Been, Identified]",Civilians Killed In Single US Airstrike Have Been Identified,"[Civilians, Killed, In, Single, US, Airstrike, Have, Been, Identified]"
4,4,Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery,1,"[Iranian, woman, jailed, for, fictional, unpublished, story, about, woman, stoned, to, death, fo...",Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery,"[Iranian, woman, jailed, for, fictional, unpublished, story, about, woman, stoned, to, death, fo..."


# Converting all Characters to Lowercase

In [18]:
#mengubah semua huruf besar menjadi huruf kecil dan ditampung pada atribut baru "lower"
data3['lower'] = data3['tokenized'].apply(lambda x: [word.lower() for word in x])
data3.head()

,id,title,label,no_contract,title_str,tokenized,lower
0,0,House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It,1,"[House, Dem, Aide:, We, did not, Even, See, Comey’s, Letter, Until, Jason, Chaffetz, Tweeted, It]",House Dem Aide: We did not Even See Comey’s Letter Until Jason Chaffetz Tweeted It,"[House, Dem, Aide, :, We, did, not, Even, See, Comey, ’, s, Letter, Until, Jason, Chaffetz, Twee...","[house, dem, aide, :, we, did, not, even, see, comey, ’, s, letter, until, jason, chaffetz, twee..."
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart",0,"[FLYNN:, Hillary, Clinton,, Big, Woman, on, Campus, -, Breitbart]","FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart","[FLYNN, :, Hillary, Clinton, ,, Big, Woman, on, Campus, -, Breitbart]","[flynn, :, hillary, clinton, ,, big, woman, on, campus, -, breitbart]"
2,2,Why the Truth Might Get You Fired,1,"[Why, the, Truth, Might, Get, You, Fired]",Why the Truth Might Get You Fired,"[Why, the, Truth, Might, Get, You, Fired]","[why, the, truth, might, get, you, fired]"
3,3,Civilians Killed In Single US Airstrike Have Been Identified,1,"[Civilians, Killed, In, Single, US, Airstrike, Have, Been, Identified]",Civilians Killed In Single US Airstrike Have Been Identified,"[Civilians, Killed, In, Single, US, Airstrike, Have, Been, Identified]","[civilians, killed, in, single, us, airstrike, have, been, identified]"
4,4,Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery,1,"[Iranian, woman, jailed, for, fictional, unpublished, story, about, woman, stoned, to, death, fo...",Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery,"[Iranian, woman, jailed, for, fictional, unpublished, story, about, woman, stoned, to, death, fo...","[iranian, woman, jailed, for, fictional, unpublished, story, about, woman, stoned, to, death, fo..."


# Removing Punctuations

In [19]:
#menghilangkan tanda baca dan ditampung oleh atribut baru yaitu "no_punc"
punc = string.punctuation
data3['no_punc'] = data3['lower'].apply(lambda x: [word for word in x if word not in punc])
data3.head()

,id,title,label,no_contract,title_str,tokenized,lower,no_punc
0,0,House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It,1,"[House, Dem, Aide:, We, did not, Even, See, Comey’s, Letter, Until, Jason, Chaffetz, Tweeted, It]",House Dem Aide: We did not Even See Comey’s Letter Until Jason Chaffetz Tweeted It,"[House, Dem, Aide, :, We, did, not, Even, See, Comey, ’, s, Letter, Until, Jason, Chaffetz, Twee...","[house, dem, aide, :, we, did, not, even, see, comey, ’, s, letter, until, jason, chaffetz, twee...","[house, dem, aide, we, did, not, even, see, comey, ’, s, letter, until, jason, chaffetz, tweeted..."
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart",0,"[FLYNN:, Hillary, Clinton,, Big, Woman, on, Campus, -, Breitbart]","FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart","[FLYNN, :, Hillary, Clinton, ,, Big, Woman, on, Campus, -, Breitbart]","[flynn, :, hillary, clinton, ,, big, woman, on, campus, -, breitbart]","[flynn, hillary, clinton, big, woman, on, campus, breitbart]"
2,2,Why the Truth Might Get You Fired,1,"[Why, the, Truth, Might, Get, You, Fired]",Why the Truth Might Get You Fired,"[Why, the, Truth, Might, Get, You, Fired]","[why, the, truth, might, get, you, fired]","[why, the, truth, might, get, you, fired]"
3,3,Civilians Killed In Single US Airstrike Have Been Identified,1,"[Civilians, Killed, In, Single, US, Airstrike, Have, Been, Identified]",Civilians Killed In Single US Airstrike Have Been Identified,"[Civilians, Killed, In, Single, US, Airstrike, Have, Been, Identified]","[civilians, killed, in, single, us, airstrike, have, been, identified]","[civilians, killed, in, single, us, airstrike, have, been, identified]"
4,4,Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery,1,"[Iranian, woman, jailed, for, fictional, unpublished, story, about, woman, stoned, to, death, fo...",Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery,"[Iranian, woman, jailed, for, fictional, unpublished, story, about, woman, stoned, to, death, fo...","[iranian, woman, jailed, for, fictional, unpublished, story, about, woman, stoned, to, death, fo...","[iranian, woman, jailed, for, fictional, unpublished, story, about, woman, stoned, to, death, fo..."


# Removing Stopwords

In [20]:
#menghilangkan kata 
stop_words = set(stopwords.words('english'))
data3['stopwords_removed'] = data3['no_punc'].apply(lambda x: [word for word in x if word not in stop_words])
data3.head()

,id,title,label,no_contract,title_str,tokenized,lower,no_punc,stopwords_removed
0,0,House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It,1,"[House, Dem, Aide:, We, did not, Even, See, Comey’s, Letter, Until, Jason, Chaffetz, Tweeted, It]",House Dem Aide: We did not Even See Comey’s Letter Until Jason Chaffetz Tweeted It,"[House, Dem, Aide, :, We, did, not, Even, See, Comey, ’, s, Letter, Until, Jason, Chaffetz, Twee...","[house, dem, aide, :, we, did, not, even, see, comey, ’, s, letter, until, jason, chaffetz, twee...","[house, dem, aide, we, did, not, even, see, comey, ’, s, letter, until, jason, chaffetz, tweeted...","[house, dem, aide, even, see, comey, ’, letter, jason, chaffetz, tweeted]"
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart",0,"[FLYNN:, Hillary, Clinton,, Big, Woman, on, Campus, -, Breitbart]","FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart","[FLYNN, :, Hillary, Clinton, ,, Big, Woman, on, Campus, -, Breitbart]","[flynn, :, hillary, clinton, ,, big, woman, on, campus, -, breitbart]","[flynn, hillary, clinton, big, woman, on, campus, breitbart]","[flynn, hillary, clinton, big, woman, campus, breitbart]"
2,2,Why the Truth Might Get You Fired,1,"[Why, the, Truth, Might, Get, You, Fired]",Why the Truth Might Get You Fired,"[Why, the, Truth, Might, Get, You, Fired]","[why, the, truth, might, get, you, fired]","[why, the, truth, might, get, you, fired]","[truth, might, get, fired]"
3,3,Civilians Killed In Single US Airstrike Have Been Identified,1,"[Civilians, Killed, In, Single, US, Airstrike, Have, Been, Identified]",Civilians Killed In Single US Airstrike Have Been Identified,"[Civilians, Killed, In, Single, US, Airstrike, Have, Been, Identified]","[civilians, killed, in, single, us, airstrike, have, been, identified]","[civilians, killed, in, single, us, airstrike, have, been, identified]","[civilians, killed, single, us, airstrike, identified]"
4,4,Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery,1,"[Iranian, woman, jailed, for, fictional, unpublished, story, about, woman, stoned, to, death, fo...",Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery,"[Iranian, woman, jailed, for, fictional, unpublished, story, about, woman, stoned, to, death, fo...","[iranian, woman, jailed, for, fictional, unpublished, story, about, woman, stoned, to, death, fo...","[iranian, woman, jailed, for, fictional, unpublished, story, about, woman, stoned, to, death, fo...","[iranian, woman, jailed, fictional, unpublished, story, woman, stoned, death, adultery]"


# Stemming vs Lemmatization

In [21]:
#menerapkan metode part of speech tags dan ditampung atribut baru "pos_tags"
data3['pos_tags'] = data3['stopwords_removed'].apply(nltk.tag.pos_tag)
data3.head()

,id,title,label,no_contract,title_str,tokenized,lower,no_punc,stopwords_removed,pos_tags
0,0,House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It,1,"[House, Dem, Aide:, We, did not, Even, See, Comey’s, Letter, Until, Jason, Chaffetz, Tweeted, It]",House Dem Aide: We did not Even See Comey’s Letter Until Jason Chaffetz Tweeted It,"[House, Dem, Aide, :, We, did, not, Even, See, Comey, ’, s, Letter, Until, Jason, Chaffetz, Twee...","[house, dem, aide, :, we, did, not, even, see, comey, ’, s, letter, until, jason, chaffetz, twee...","[house, dem, aide, we, did, not, even, see, comey, ’, s, letter, until, jason, chaffetz, tweeted...","[house, dem, aide, even, see, comey, ’, letter, jason, chaffetz, tweeted]","[(house, NN), (dem, NN), (aide, RB), (even, RB), (see, VB), (comey, JJ), (’, NNP), (letter, NN),..."
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart",0,"[FLYNN:, Hillary, Clinton,, Big, Woman, on, Campus, -, Breitbart]","FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart","[FLYNN, :, Hillary, Clinton, ,, Big, Woman, on, Campus, -, Breitbart]","[flynn, :, hillary, clinton, ,, big, woman, on, campus, -, breitbart]","[flynn, hillary, clinton, big, woman, on, campus, breitbart]","[flynn, hillary, clinton, big, woman, campus, breitbart]","[(flynn, JJ), (hillary, JJ), (clinton, NN), (big, JJ), (woman, NN), (campus, NN), (breitbart, NN)]"
2,2,Why the Truth Might Get You Fired,1,"[Why, the, Truth, Might, Get, You, Fired]",Why the Truth Might Get You Fired,"[Why, the, Truth, Might, Get, You, Fired]","[why, the, truth, might, get, you, fired]","[why, the, truth, might, get, you, fired]","[truth, might, get, fired]","[(truth, NN), (might, MD), (get, VB), (fired, VBN)]"
3,3,Civilians Killed In Single US Airstrike Have Been Identified,1,"[Civilians, Killed, In, Single, US, Airstrike, Have, Been, Identified]",Civilians Killed In Single US Airstrike Have Been Identified,"[Civilians, Killed, In, Single, US, Airstrike, Have, Been, Identified]","[civilians, killed, in, single, us, airstrike, have, been, identified]","[civilians, killed, in, single, us, airstrike, have, been, identified]","[civilians, killed, single, us, airstrike, identified]","[(civilians, NNS), (killed, VBN), (single, JJ), (us, PRP), (airstrike, IN), (identified, VBN)]"
4,4,Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery,1,"[Iranian, woman, jailed, for, fictional, unpublished, story, about, woman, stoned, to, death, fo...",Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery,"[Iranian, woman, jailed, for, fictional, unpublished, story, about, woman, stoned, to, death, fo...","[iranian, woman, jailed, for, fictional, unpublished, story, about, woman, stoned, to, death, fo...","[iranian, woman, jailed, for, fictional, unpublished, story, about, woman, stoned, to, death, fo...","[iranian, woman, jailed, fictional, unpublished, story, woman, stoned, death, adultery]","[(iranian, JJ), (woman, NN), (jailed, VBD), (fictional, JJ), (unpublished, JJ), (story, NN), (wo..."


In [22]:
#mengganti part of speech tags 
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
data3['wordnet_pos'] = data3['pos_tags'].apply(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])

In [23]:
data3.head()

,id,title,label,no_contract,title_str,tokenized,lower,no_punc,stopwords_removed,pos_tags,wordnet_pos
0,0,House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It,1,"[House, Dem, Aide:, We, did not, Even, See, Comey’s, Letter, Until, Jason, Chaffetz, Tweeted, It]",House Dem Aide: We did not Even See Comey’s Letter Until Jason Chaffetz Tweeted It,"[House, Dem, Aide, :, We, did, not, Even, See, Comey, ’, s, Letter, Until, Jason, Chaffetz, Twee...","[house, dem, aide, :, we, did, not, even, see, comey, ’, s, letter, until, jason, chaffetz, twee...","[house, dem, aide, we, did, not, even, see, comey, ’, s, letter, until, jason, chaffetz, tweeted...","[house, dem, aide, even, see, comey, ’, letter, jason, chaffetz, tweeted]","[(house, NN), (dem, NN), (aide, RB), (even, RB), (see, VB), (comey, JJ), (’, NNP), (letter, NN),...","[(house, n), (dem, n), (aide, r), (even, r), (see, v), (comey, a), (’, n), (letter, n), (jason, ..."
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart",0,"[FLYNN:, Hillary, Clinton,, Big, Woman, on, Campus, -, Breitbart]","FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart","[FLYNN, :, Hillary, Clinton, ,, Big, Woman, on, Campus, -, Breitbart]","[flynn, :, hillary, clinton, ,, big, woman, on, campus, -, breitbart]","[flynn, hillary, clinton, big, woman, on, campus, breitbart]","[flynn, hillary, clinton, big, woman, campus, breitbart]","[(flynn, JJ), (hillary, JJ), (clinton, NN), (big, JJ), (woman, NN), (campus, NN), (breitbart, NN)]","[(flynn, a), (hillary, a), (clinton, n), (big, a), (woman, n), (campus, n), (breitbart, n)]"
2,2,Why the Truth Might Get You Fired,1,"[Why, the, Truth, Might, Get, You, Fired]",Why the Truth Might Get You Fired,"[Why, the, Truth, Might, Get, You, Fired]","[why, the, truth, might, get, you, fired]","[why, the, truth, might, get, you, fired]","[truth, might, get, fired]","[(truth, NN), (might, MD), (get, VB), (fired, VBN)]","[(truth, n), (might, n), (get, v), (fired, v)]"
3,3,Civilians Killed In Single US Airstrike Have Been Identified,1,"[Civilians, Killed, In, Single, US, Airstrike, Have, Been, Identified]",Civilians Killed In Single US Airstrike Have Been Identified,"[Civilians, Killed, In, Single, US, Airstrike, Have, Been, Identified]","[civilians, killed, in, single, us, airstrike, have, been, identified]","[civilians, killed, in, single, us, airstrike, have, been, identified]","[civilians, killed, single, us, airstrike, identified]","[(civilians, NNS), (killed, VBN), (single, JJ), (us, PRP), (airstrike, IN), (identified, VBN)]","[(civilians, n), (killed, v), (single, a), (us, n), (airstrike, n), (identified, v)]"
4,4,Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery,1,"[Iranian, woman, jailed, for, fictional, unpublished, story, about, woman, stoned, to, death, fo...",Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery,"[Iranian, woman, jailed, for, fictional, unpublished, story, about, woman, stoned, to, death, fo...","[iranian, woman, jailed, for, fictional, unpublished, story, about, woman, stoned, to, death, fo...","[iranian, woman, jailed, for, fictional, unpublished, story, about, woman, stoned, to, death, fo...","[iranian, woman, jailed, fictional, unpublished, story, woman, stoned, death, adultery]","[(iranian, JJ), (woman, NN), (jailed, VBD), (fictional, JJ), (unpublished, JJ), (story, NN), (wo...","[(iranian, a), (woman, n), (jailed, v), (fictional, a), (unpublished, a), (story, n), (woman, n)..."


In [24]:
#mengembalikan kata ke bentuk dasar
wnl = WordNetLemmatizer()
data3['lemmatized'] = data3['wordnet_pos'].apply(lambda x: [wnl.lemmatize(word, tag) for word, tag in x])

In [25]:
data3.head()

,id,title,label,no_contract,title_str,tokenized,lower,no_punc,stopwords_removed,pos_tags,wordnet_pos,lemmatized
0,0,House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It,1,"[House, Dem, Aide:, We, did not, Even, See, Comey’s, Letter, Until, Jason, Chaffetz, Tweeted, It]",House Dem Aide: We did not Even See Comey’s Letter Until Jason Chaffetz Tweeted It,"[House, Dem, Aide, :, We, did, not, Even, See, Comey, ’, s, Letter, Until, Jason, Chaffetz, Twee...","[house, dem, aide, :, we, did, not, even, see, comey, ’, s, letter, until, jason, chaffetz, twee...","[house, dem, aide, we, did, not, even, see, comey, ’, s, letter, until, jason, chaffetz, tweeted...","[house, dem, aide, even, see, comey, ’, letter, jason, chaffetz, tweeted]","[(house, NN), (dem, NN), (aide, RB), (even, RB), (see, VB), (comey, JJ), (’, NNP), (letter, NN),...","[(house, n), (dem, n), (aide, r), (even, r), (see, v), (comey, a), (’, n), (letter, n), (jason, ...","[house, dem, aide, even, see, comey, ’, letter, jason, chaffetz, tweet]"
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart",0,"[FLYNN:, Hillary, Clinton,, Big, Woman, on, Campus, -, Breitbart]","FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart","[FLYNN, :, Hillary, Clinton, ,, Big, Woman, on, Campus, -, Breitbart]","[flynn, :, hillary, clinton, ,, big, woman, on, campus, -, breitbart]","[flynn, hillary, clinton, big, woman, on, campus, breitbart]","[flynn, hillary, clinton, big, woman, campus, breitbart]","[(flynn, JJ), (hillary, JJ), (clinton, NN), (big, JJ), (woman, NN), (campus, NN), (breitbart, NN)]","[(flynn, a), (hillary, a), (clinton, n), (big, a), (woman, n), (campus, n), (breitbart, n)]","[flynn, hillary, clinton, big, woman, campus, breitbart]"
2,2,Why the Truth Might Get You Fired,1,"[Why, the, Truth, Might, Get, You, Fired]",Why the Truth Might Get You Fired,"[Why, the, Truth, Might, Get, You, Fired]","[why, the, truth, might, get, you, fired]","[why, the, truth, might, get, you, fired]","[truth, might, get, fired]","[(truth, NN), (might, MD), (get, VB), (fired, VBN)]","[(truth, n), (might, n), (get, v), (fired, v)]","[truth, might, get, fire]"
3,3,Civilians Killed In Single US Airstrike Have Been Identified,1,"[Civilians, Killed, In, Single, US, Airstrike, Have, Been, Identified]",Civilians Killed In Single US Airstrike Have Been Identified,"[Civilians, Killed, In, Single, US, Airstrike, Have, Been, Identified]","[civilians, killed, in, single, us, airstrike, have, been, identified]","[civilians, killed, in, single, us, airstrike, have, been, identified]","[civilians, killed, single, us, airstrike, identified]","[(civilians, NNS), (killed, VBN), (single, JJ), (us, PRP), (airstrike, IN), (identified, VBN)]","[(civilians, n), (killed, v), (single, a), (us, n), (airstrike, n), (identified, v)]","[civilian, kill, single, u, airstrike, identify]"
4,4,Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery,1,"[Iranian, woman, jailed, for, fictional, unpublished, story, about, woman, stoned, to, death, fo...",Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery,"[Iranian, woman, jailed, for, fictional, unpublished, story, about, woman, stoned, to, death, fo...","[iranian, woman, jailed, for, fictional, unpublished, story, about, woman, stoned, to, death, fo...","[iranian, woman, jailed, for, fictional, unpublished, story, about, woman, stoned, to, death, fo...","[iranian, woman, jailed, fictional, unpublished, story, woman, stoned, death, adultery]","[(iranian, JJ), (woman, NN), (jailed, VBD), (fictional, JJ), (unpublished, JJ), (story, NN), (wo...","[(iranian, a), (woman, n), (jailed, v), (fictional, a), (unpublished, a), (story, n), (woman, n)...","[iranian, woman, jail, fictional, unpublished, story, woman, stone, death, adultery]"
